# Uncertainties & Intervals

### Monte Carlo Simulations

Monte Carlo simulation is a technique used to study how a model  responds to randomly generated inputs. It typically involves a  three-step process: 
1. Randomly generate “N” inputs (sometimes called scenarios).
2. Run a simulation for each of the “N” inputs. Simulations are run on a computerized model of the system being analyzed.
3. Aggregate and assess the outputs from the simulations. Common  measures include the mean value of an output, the distribution of output values, and  the minimum or maximum output value.

When are Monte Carlo Simulations needed?
- Computing *intervals* and *bounds* in signals, when the parameters are uncertain.
- *Creating scenarios* for solving various selection problems which cannot be solved by integer or mixed integer programming.

### Outline

In this notebook we demonstrate how to run (parallel) Monte Carlo Simulations in EPyT-Flow for investigating how uncertainty in the base demands affects the pressures in a network.

This is useful for:
- Fault diagnosis
- State estimation
- Forecasting
- Sensitivity studies
- Demonstrating robustness of solutions
- ...

We demonstrate how to:
1. Setup the scenarios
2. Run the Monte Carlo Simulation
3. Analyze the results
4. Run the Monte Carlo Simulation in parallel

In [ ]:
%pip install epyt-flow

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=ImportWarning)

import numpy as np
import matplotlib.pyplot as plt

from epyt_flow.simulation import ScenarioSimulator, ParallelScenarioSimulation, ScadaData
from epyt_flow.uncertainty import ModelUncertainty, UniformUncertainty
from epyt_flow.utils import plot_timeseries_data

### 1. Setup

Prepare the Monte Carlo simulation -- i.e. general parameters and sensor configuration:

In [ ]:
# Number of simulations
n_sim = 100

# 5% max uncertainty in base demands
eta_bar = 0.05

We implement the base demand uncertainty by utilizing the EPyT-Flow module on [uncertainties](https://epyt-flow.readthedocs.io/en/stable/tut.uncertainty.html).
That is, we derive a new class `MyBaseDemandUncertainty` for implementing the uncertainty logic and (later on) specifying it as part of the [model uncertainty](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.set_model_uncertainty):

In [ ]:
# Specify and implement the base demand uncertainty
# delta = base_demand * uniform_random[-a, a]
# base_demand =  base_demand + delta
class MyBaseDemandUncertainty(UniformUncertainty):
    def __init__(self, **kwds):
        super().__init__(**kwds)

    def apply(self, data: float) -> float:
        z = data * np.random.uniform(low=self.low, high=self.high)
        return data + z

In [ ]:
# Specify uncertainty
base_demand_uncertainty = MyBaseDemandUncertainty(low=-eta_bar, high=eta_bar)

### 2. Run the Monte Carlo simulation:

In [ ]:
# Run Monte Carlo simulation
mcs_results_pressure = []
mcs_results_quality = []

for i in range(n_sim):
    # Create scenario based on Net2
    with ScenarioSimulator(f_inp_in="Net2.inp") as sim:
        """
        # TODO: Do it without the model uncertainty class
        # Compute and set new base demands
        base_demands = sim.epanet_api.getNodeBaseDemands()[1]
        delta_bd = (2*np.random.rand(len(base_demands))-1) * eta_bar * base_demands
        new_base_demands = base_demands + delta_bd
        #print(base_demands)
        #print(new_base_demands)

        sim.epanet_api.setNodeBaseDemands(new_base_demands)
        #"""
        sim.set_model_uncertainty(ModelUncertainty(base_demand_uncertainty=base_demand_uncertainty))

        # Place pressure sensors at each node
        sim.set_pressure_sensors(sim.sensor_config.nodes)

        # Place quality sensors at each node
        sim.set_node_quality_sensors(sim.sensor_config.nodes)

        # Run simulation and retrieve pressures and quality at each node
        scada_data = sim.run_simulation()

        #plot_timeseries_data(scada_data.get_data_pressures(["5"]).T)
        mcs_results_pressure.append(scada_data.get_data_pressures().T)  # Transpose: Each row contains one tim series! 
        mcs_results_quality.append(scada_data.get_data_nodes_quality().T)

# Create NumPy array
mcs_results_pressure = np.array(mcs_results_pressure)
mcs_results_quality = np.array(mcs_results_quality)

#### 3. Analyze the results

How does the pressure fluctuates under the uncertain base demands?

In [ ]:
node_idx = 4   # Investigate the pressure at the fifth node -- refers to node "5", recall that indicies start at zero!
pressure_at_node = mcs_results_pressure[:, node_idx]

In [ ]:
plot_timeseries_data(pressure_at_node,
                     x_axis_label="Time steps (1min)",
                     y_axis_label="Pressure in $psi$")

Compute upper and lower bounds:

In [ ]:
upper_bound = np.max(pressure_at_node, axis=0)
lower_bound = np.min(pressure_at_node, axis=0)
average, var = np.mean(pressure_at_node, axis=0), np.var(pressure_at_node, axis=0)

upper_bound, lower_bound, average, var

In [ ]:
_, ax = plt.subplots()
ax.plot(upper_bound, label="Upper bound")
ax.plot(lower_bound, label="Lower bound")
ax.plot(average, label="Average")
ax.legend()
ax.set_xlabel("Time steps (1min)")
ax.set_ylabel("Pressure in $psi$")

### 4. Parallel Computations (Advanced topic)

EPyT-Flow supports the simulation of multiple scenarios in parallel -- see [`ParallelScenarioSimulation`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.parallel_simulation.ParallelScenarioSimulation) for details -- which we are going to utilize to speed up the Monte Carlo simulation.

Create scenarios:

In [ ]:
mcs_sceanrios = []

for i in range(n_sim):
    # Create sceanrio based on Net2
    with ScenarioSimulator(f_inp_in="Net2.inp") as sim:
        # Specify base demand uncertainty
        base_demand_uncertainty = MyBaseDemandUncertainty(low=-eta_bar, high=eta_bar)
        sim.set_model_uncertainty(ModelUncertainty(base_demand_uncertainty=base_demand_uncertainty))

        # Specify sensor configuration
        sim.set_pressure_sensors(sim.sensor_config.nodes)
        sim.set_node_quality_sensors(sim.sensor_config.nodes)

        # Export scenario configuration
        sim.save_to_epanet_file(f"Net2_{i}.inp") # EPyT can not load the same .inp files more than once at the same time!
        mcs_sceanrios.append(sim.get_scenario_config())

Run Monte Carlo simulation in parallel using up to 4 CPU cores:

In [ ]:
# Callback handler returns the pressure readings of the node "5"
def __callback(scada_data: ScadaData, _, scenario_idx: int) -> np.ndarray:
    return scada_data.get_data_pressures(["5"])

# Run simulations in parallel
msc_results_pressure = ParallelScenarioSimulation.run(scenarios=mcs_sceanrios, callback=__callback, n_jobs=4)
msc_results_pressure = np.array(msc_results_pressure)

Evaluate pressure fluctuations:

In [ ]:
plot_timeseries_data(pressure_at_node,
                     x_axis_label="Time steps (1min)",
                     y_axis_label="Pressure in $psi$")

In [ ]:
upper_bound = np.max(pressure_at_node, axis=0)
lower_bound = np.min(pressure_at_node, axis=0)
average, var = np.mean(pressure_at_node, axis=0), np.var(pressure_at_node, axis=0)

In [ ]:
upper_bound, lower_bound, average, var